# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [1]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [2]:
# Use this cell for any import statements you add
import subprocess
import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [3]:
# 'artists' dictionary stores artists Rascal Flatts and Kelly Clarkson on AZlyrics
artists = {'rascal':"https://www.azlyrics.com/r/rascalflatts.html",
           'clarkson':"https://www.azlyrics.com/c/clarkson.html"} 

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: According to 'https://www.azlyrics.com/robots.txt', scraping of the page is allowed for by this page and is denoted by the 'user=agent: *' and 'Allow: /'.  The '/lyricsdb/' and '/song/' directories disallowed for scraping by this page.


In [4]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page by parsing the html
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links.
    for link in soup.select('div.listalbum-item > a[href]'):
        href = link['href']
        full_url = "https://www.azlyrics.com" + href[2:] if href.startswith("..") else href
        lyrics_pages[artist].append(full_url)


Let's make sure we have enough lyrics pages to scrape. 

In [5]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [6]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For rascal we have 176.
The full pull will take for this artist will take 0.49 hours.
For clarkson we have 255.
The full pull will take for this artist will take 0.71 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [7]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [8]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality
if not os.path.exists("lyrics"):
    os.makedirs("lyrics")
    with open(os.path.join("lyrics", ".gitkeep"), "w") as f:
        f.write("")

# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.
if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [9]:
#used chatgpt on 5/11/2025 to clean code and fix errors
url = "https://www.azlyrics.com" 
start = time.time()
headers = {'User-Agent': 'Mozilla/5.0'}
total_pages = 0 

def generate_filename_from_url(url):
    return url.strip('/').split('/')[-1].replace('.html', '') + '.txt' #generates clean title of song and appends .txt

for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    artist_folder = os.path.join("lyrics", artist.lower())
    os.makedirs(artist_folder, exist_ok=True)
    
   # 2. Iterate over the lyrics pages
    for link in lyrics_pages[artist][:40]:  #pulling first 40 songs of each artist
        try:
            
            # 3. Request the lyrics page
            time.sleep(5 + 10 * random.random())
            full_url = link if link.startswith("http") else url + link
            response = requests.get(full_url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # 4. Extract the title and lyrics from the page
            title = soup.find("title").text.strip()
            divs = soup.find_all("div", class_=None, id=None)
            lyrics = divs[0].get_text(separator='\n').strip() if divs else "[Lyrics not found]"

            # 5. Write out the title and lyrics
            filename = generate_filename_from_url(link)
            filepath = os.path.join(artist_folder, filename)

            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(title + "\n\n" + lyrics)

            total_pages += 1
            print(f"Saved: {filepath}")

        except Exception as e:
            print(f"Error with {link}: {e}")

end = time.time()
print(f"\n Scraped {total_pages} pages in {round(end - start, 2)} seconds.")
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
    

Saved: lyrics/rascal/prayinfordaylight.txt
Saved: lyrics/rascal/thiseverydaylove.txt
Saved: lyrics/rascal/whileyoulovedme.txt
Saved: lyrics/rascal/somesay.txt
Saved: lyrics/rascal/seemethrough.txt
Saved: lyrics/rascal/onegoodlove.txt
Saved: lyrics/rascal/itsnotjustme.txt
Saved: lyrics/rascal/waitingallmylife.txt
Saved: lyrics/rascal/fromtimetotime.txt
Saved: lyrics/rascal/longslowbeautifuldance.txt
Saved: lyrics/rascal/immovingon.txt
Saved: lyrics/rascal/thesedays.txt
Saved: lyrics/rascal/toogoodistrue.txt
Saved: lyrics/rascal/imelt.txt
Saved: lyrics/rascal/mayberry.txt
Saved: lyrics/rascal/loveyououtloud.txt
Saved: lyrics/rascal/drycountygirl.txt
Saved: lyrics/rascal/likeiam.txt
Saved: lyrics/rascal/you.txt
Saved: lyrics/rascal/fallinupsidedown.txt
Saved: lyrics/rascal/shineon.txt
Saved: lyrics/rascal/myworstfear.txt
Saved: lyrics/rascal/whereyouare.txt
Saved: lyrics/rascal/blessthebrokenroad.txt
Saved: lyrics/rascal/thenidid.txt
Saved: lyrics/rascal/feelsliketoday.txt
Saved: lyrics/r

In [10]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.24 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [11]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [12]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For clarkson we have 40 files.
For clarkson we have roughly 12852 words, 1067 are unique.
For rascal we have 40 files.
For rascal we have roughly 11500 words, 1188 are unique.
